In [117]:
#SCRAPE THE WEBSITE AND BUILD A PANDAS TABLE - RUN THIS ONE TIME UNLESS "TABLE" IS NOT DEFINED

import requests
from bs4 import BeautifulSoup

def get_schedule():
    print("Gathering Schedule Data...")
    # Make a request to the website
    r = requests.get('https://www.fftoday.com/nfl/schedule.php')
    r_html = r.text

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(r_html, 'html.parser')

    # Find the table in the HTML
    table = soup.find('table', attrs={'width': '80%', 'border': '0', 'cellpadding': '0', 'cellspacing': '0'})

    # Find all rows in the table with a white background
    rows = table.find_all('tr', attrs={'bgcolor': '#ffffff'})
    print("Schedule Data Retrieved")
    return table, rows
    #print(table)


In [ ]:
#THIS CELL FINDS ALL THE DAT ABOUT THE SCHEDULE, TRAVEL, AND RANKINGS - THEN PUTS THEM INTO THE DF

import numpy as np
from math import radians, sin, cos, sqrt, atan2
import pytz
from dateutil.parser import parse
from datetime import timedelta
import pandas as pd

def collect_schedule_travel_ranking_data(pd):
    data = []
    # Initialize a variable to hold the last valid date and week
    last_date = None
    start_date_str = 'September 4, 2024'
    start_date = parse(start_date_str)
    week = 1
    # Initialize a dictionary to store the last game date for each team
    last_game = {}
    last_away_game = {}
    # Initialize dictionaries to store cumulative rest advantage for each team
    cumulative_advantage = {}
    # 0: Stadium | 1: Lattitude | 2: Longitude | 3: Timezone | 4: Division | 5: Start of 2023 Season Rank | 6: Current Rank | 7: Average points better than Average Team (Used for Spread and Odds Calculation)
    stadiums = {
        'Arizona Cardinals': ['State Farm Stadium', 33.5277, -112.262608, 'America/Denver', 'NFC West', 26, 24, -.5],
        'Atlanta Falcons': ['Mercedez-Benz Stadium', 33.757614, -84.400972, 'America/New_York', 'NFC South', 13, 18, 0],
        'Baltimore Ravens': ['M&T Stadium', 39.277969, -76.622767, 'America/New_York', 'AFC North', 3, 3, 3],
        'Buffalo Bills': ['Highmark Stadium', 42.773739, -78.786978, 'America/New_York', 'AFC East', 5, 7, 3.5],
        'Carolina Panthers': ['Bank of America Stadium', 35.225808, -80.852861, 'America/New_York', 'NFC South', 32, 32, -7],
        'Chicago Bears': ['Soldier Field', 41.862306, -87.616672, 'America/Chicago', 'NFC North', 15, 19, -2],
        'Cincinnati Bengals': ['Paycor Stadium', 39.095442, -84.516039, 'America/New_York', 'AFC North', 6, 11, 2],
        'Cleveland Browns': ['Cleveland Browns Stadium', 41.506022, -81.699564, 'America/New_York', 'AFC North', 17, 20, 0],
        'Dallas Cowboys': ['AT&T Stadium', 32.747778, -97.092778, 'America/Chicago', 'NFC East', 9, 6, 1.5],
        'Denver Broncos': ['Empower Field at Mile High', 39.743936, -105.020097, 'America/Denver', 'AFC West', 29, 29, -5.5],
        'Detroit Lions': ['Ford Field', 42.340156, -83.045808, 'America/New_York', 'NFC North', 4, 5, 3],
        'Green Bay Packers': ['Lambeau Field', 44.501306, -88.062167, 'America/Chicago', 'NFC North', 10, 12, -4],
        'Houston Texans': ['NRG Stadium', 29.684781, -95.410956, 'America/Chicago', 'AFC South', 7, 8, 3.5],
        'Indianapolis Colts': ['Lucas Oil Stadium', 39.760056, -86.163806, 'America/New_York', 'AFC South', 20, 19, -2],
        'Jacksonville Jaguars': ['Everbank Stadium', 30.323925, -81.637356, 'America/New_York', 'AFC South', 18, 17, -.5],
        'Kansas City Chiefs': ['Arrowhead Stadium', 39.048786, -94.484566, 'America/Chicago', 'AFC West', 1, 1, 5],
        'Las Vegas Raiders': ['Allegiant Stadium', 36.090794, -115.183952, 'America/Los_Angeles', 'AFC West', 28, 26, -3],
        'Los Angeles Chargers': ['SoFi Stadium', 33.953587, -118.33963, 'America/Los_Angeles', 'AFC West', 14, 17, 1.5],
        'Los Angeles Rams': ['SoFi Stadium', 33.953587, -118.33963, 'America/Los_Angeles', 'NFC West', 16, 15, -3.5],
        'Miami Dolphins': ['Hard Rock Stadium', 25.957919, -80.238842, 'America/New_York', 'AFC East', 12, 11, -4],
        'Minnesota Vikings': ['U.S Bank Stadium', 44.973881, -93.258094, 'America/Chicago', 'NFC North', 24, 22, .5],
        'New England Patriots': ['Gillette Stadium', 42.090925, -71.26435, 'America/New_York', 'AFC East', 31, 27, -4.5],
        'New Orleans Saints': ['Caesars Superdome', 29.950931, -90.081364, 'America/Chicago', 'NFC South', 23, 16, 2],
        'New York Giants': ['MetLife Stadium', 40.812194, -74.076983, 'America/New_York', 'NFC East', 27, 31, -5],
        'New York Jets': ['MetLife Stadium', 40.812194, -74.076983, 'America/New_York', 'AFC East', 11, 13, 1],
        'Philadelphia Eagles': ['Lincoln Financial Field', 39.900775, -75.167453, 'America/New_York', 'NFC East', 8, 4, 3],
        'Pittsburgh Steelers': ['Acrisure Stadium', 40.446786, -80.015761, 'America/New_York', 'AFC North', 19, 16, .5],
        'San Francisco 49ers': ['Levi\'s Stadium', 37.713486, -122.386256, 'America/Los_Angeles', 'NFC West', 2, 1.5, 4.5],
        'Seattle Seahawks': ['Lumen Field', 47.595153, -122.331625, 'America/Los_Angeles', 'NFC West', 22, 19, .5],
        'Tampa Bay Buccaneers': ['Raymomd James Stadium', 27.975967, -82.50335, 'America/New_York', 'NFC South', 21, 21, 0],
        'Tennessee Titans': ['Nissan Stadium', 36.166461, -86.771289, 'America/Chicago', 'AFC South', 20, 24, -2.5],
        'Washington Commanders': ['FedExField', 38.907697, -76.864517, 'America/New_York', 'NFC East', 25, 28, -3.5]
    }

    #Get the distances traveled
    def haversine(lat1, lon1, lat2, lon2):
        # Convert degrees to radians
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Differences
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        # Haversine formula
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        # Radius of earth in kilometers. Use 3956 for miles
        r = 3956

        return c * r

    #Get the timzone differences
    def calculate_hours_difference(tz1, tz2):
        # This function calculates the difference in hours between two timezones
        tz1_offset = pytz.timezone(tz1).utcoffset(pd.Timestamp.now()).total_seconds() / 3600
        tz2_offset = pytz.timezone(tz2).utcoffset(pd.Timestamp.now()).total_seconds() / 3600
        return tz1_offset - tz2_offset

    # Loop through each row in the table
    for schedule_row in schedule_table.find_all('tr'):
        # Check if the row has a white background
        if schedule_row.get('bgcolor') == '#ffffff':
            # Find all columns in the row
            cols = schedule_row.find_all('td')
    #        print(cols)
            # Get the text from each column and strip leading/trailing whitespaces
            cols_text = []
            actual_stadium = []
            for col in cols:
                text = BeautifulSoup(col.get_text(strip=True), 'html.parser').text
                if " ¹" in text:
                    actual_stadium = "London, UK"
                    text = text.replace(" ¹", "")
                else:
                    actual_stadium = ''
                if " *" in text:
                    text = text.replace(" *", "")
                cols_text.append(text)
            # If the date field is not blank, update last_date and check if it's a new week
            if cols_text[0].strip() != '':
                # Parse the date
                date_str = cols_text[0]
                date = parse(date_str)
                if date.month == 1:
                    # Change the year to 2025
                    date = date.replace(year=2025)
                # Adjust week for games on or after November 30th
                if date >= pd.Timestamp('2024-11-30'):
                    week += 1
                # Adjust week for games on or after December 27th
                if date >= pd.Timestamp('2024-12-27'):
                    week += 1
                # Calculate the difference in days
                days_diff = (date - start_date).days

                # Calculate the week number
                week = 1 + days_diff // 7

                # Update cols_text with the week information
                #cols_text.insert(0, f'Week {week}')

                # Rest of your existing logic (rest days, advantage, etc.)

                # Update last_date
                last_date = date
            # If the date field is blank and last_date is not None, use last_date
            elif last_date is not None:
                cols_text[0] = last_date.strftime('%a %b %d')
            # Add week to the start of cols_text
            cols_text.insert(0, f'Week {week}')

            # Calculate rest days for away team and add it to cols_text
            away_team = cols_text[3]
            home_team = cols_text[4]
            if away_team in last_game:
                away_rest_days = (last_date - last_game[away_team]).days
            else:
                away_rest_days = 'NA'
            if home_team in last_game:
                home_rest_days = (last_date - last_game[home_team]).days
            else:
                home_rest_days = 'NA'   
            # Calculate rest advantage for both teams and add it to cols_text
            if isinstance(away_rest_days, int) and isinstance(home_rest_days, int):
                away_advantage = away_rest_days - home_rest_days
                home_advantage = home_rest_days - away_rest_days

                # Update cumulative rest advantage for both teams regardless of whether they are home or away this game.
                cumulative_advantage[away_team] = cumulative_advantage.get(away_team, 0) + away_advantage
                cumulative_advantage[home_team] = cumulative_advantage.get(home_team, 0) + home_advantage

            else:
                away_advantage = 'NA'
                home_advantage = 'NA'        

            cols_text.extend([away_rest_days, home_rest_days, away_advantage, home_advantage,
                              cumulative_advantage.get(away_team, 'NA'), 
                              cumulative_advantage.get(home_team, 'NA')])  

            # Update last game date for both teams regardless of whether they are home or away this game.
            last_game[away_team] = last_date
            last_game[home_team] = last_date

            # Check if the current game is an away game in the next week after the last away game
            back_to_back_away = False
            if away_team in last_away_game and last_away_game[away_team] == week - 1:
                back_to_back_away = True
            # Update the last away game week for the away team
            last_away_game[away_team] = week


            data.append(cols_text + [actual_stadium, back_to_back_away])
    #        print(cols_text)
    df = pd.DataFrame(data, columns=['Week', 'Date', 'Time', 'Away Team', 'Home Team', 
                                     'Away Team Weekly Rest', 'Home Team Weekly Rest', 
                                     'Weekly Away Rest Advantage', 'Weekly Home Rest Advantage',
                                     'Away Cumulative Rest Advantage', 'Home Cumulative Rest Advantage','Actual Stadium', 'Back to Back Away Games'])

    df['Date'] = df['Date'].str.replace(r'(\w+)\s(\w+)\s(\d+)', r'\2 \3, 2024', regex=True)
    df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
    # Adjust January games to 2025 in the DataFrame
    df['Date'] = df['Date'].apply(lambda x: x.replace(year=2025) if x.month == 1 else x)
    df['Week'] = df['Week'].str.replace('Week ', '', regex=False).astype(int)

    # Increment 'Week' for games on or after 2024-11-30
    df.loc[df['Date'] >= pd.to_datetime('2024-11-30'), 'Week'] += 1
    df.loc[df['Date'] >= pd.to_datetime('2024-12-27'), 'Week'] += 1

    # Convert 'Week' back to string format if needed
    df['Week'] = 'Week ' + df['Week'].astype(str)
    df['Away Team Current Week Cumulative Rest Advantage'] = pd.to_numeric(df['Away Cumulative Rest Advantage'], errors='coerce').fillna(0) - pd.to_numeric(df['Home Cumulative Rest Advantage'], errors='coerce').fillna(0)
    df['Home Team Current Week Cumulative Rest Advantage'] = pd.to_numeric(df['Home Cumulative Rest Advantage'], errors='coerce').fillna(0) - pd.to_numeric(df['Away Cumulative Rest Advantage'], errors='coerce').fillna(0)
    df['Away Team Division'] = df['Away Team'].map(lambda team: stadiums[team][4] if team in stadiums else 'NA')
    df['Away Stadium'] = df['Away Team'].map(lambda team: stadiums[team][0] if team in stadiums else 'NA')
    df['Away Stadium Latitude'] = df['Away Team'].map(lambda team: stadiums[team][1] if team in stadiums else 'NA')
    df['Away Stadium Longitude'] = df['Away Team'].map(lambda team: stadiums[team][2] if team in stadiums else 'NA')
    df['Away Stadium TimeZone'] = df['Away Team'].map(lambda team: stadiums[team][3] if team in stadiums else 'NA')

    df['Home Team Division'] = df['Home Team'].map(lambda team: stadiums[team][4] if team in stadiums else 'NA')
    df['Home Stadium'] = df['Home Team'].map(lambda team: stadiums[team][0] if team in stadiums else 'NA')
    df['Home Stadium Latitude'] = df['Home Team'].map(lambda team: stadiums[team][1] if team in stadiums else 'NA')
    df['Home Stadium Longitude'] = df['Home Team'].map(lambda team: stadiums[team][2] if team in stadiums else 'NA')
    df['Home Stadium TimeZone'] = df['Home Team'].map(lambda team: stadiums[team][3] if team in stadiums else 'NA')
    df.loc[df['Actual Stadium'] == '', 'Actual Stadium'] = df['Home Stadium']

    #df['Home Team'] = df['Home Team'].str.replace(' *', '')
    #df.to_csv('test.csv', index=False)


    # Add new columns to the DataFrame
    df['Actual Stadium Latitude'] = np.where(df['Actual Stadium'] == 'London, UK', 51.555973, df['Home Stadium Latitude'])
    df['Actual Stadium Longitude'] = np.where(df['Actual Stadium'] == 'London, UK', -0.279672, df['Home Stadium Longitude'])
    df['Actual Stadium TimeZone'] = np.where(df['Actual Stadium'] == 'London, UK', 'Europe/London', df['Home Stadium TimeZone'])

    df['Away Stadium Latitude'] = pd.to_numeric(df['Away Stadium Latitude'])
    df['Away Stadium Longitude'] = pd.to_numeric(df['Away Stadium Longitude'])
    df['Actual Stadium Latitude'] = pd.to_numeric(df['Actual Stadium Latitude'])
    df['Actual Stadium Longitude'] = pd.to_numeric(df['Actual Stadium Longitude'])
    df['Home Stadium Latitude'] = pd.to_numeric(df['Home Stadium Latitude'])
    df['Home Stadium Longitude'] = pd.to_numeric(df['Home Stadium Longitude'])

    df['Away Travel Distance'] = df.apply(lambda row: round(haversine(row['Away Stadium Latitude'], row['Away Stadium Longitude'], row['Actual Stadium Latitude'], row['Actual Stadium Longitude'])), axis=1)
    df['Home Travel Distance'] = df.apply(lambda row: round(haversine(row['Home Stadium Latitude'], row['Home Stadium Longitude'], row['Actual Stadium Latitude'], row['Actual Stadium Longitude'])), axis=1)

    df['Away Travel Advantage'] =  df['Home Travel Distance'] - df['Away Travel Distance']
    df['Home Travel Advantage'] =  df['Away Travel Distance'] - df['Home Travel Distance']

    # Apply the function to your DataFrame
    df['Away Timezone Change'] = df.apply(lambda row: calculate_hours_difference(row['Away Stadium TimeZone'], row['Actual Stadium TimeZone']), axis=1)
    df['Home Timezone Change'] = df.apply(lambda row: calculate_hours_difference(row['Home Stadium TimeZone'], row['Actual Stadium TimeZone']), axis=1)

    # Initialize empty lists for storing last game timezones
    last_game_timezones_away = []
    last_game_timezones_home = []

    # Initialize dictionary for storing last game timezone for each team
    last_game_timezone = {}

    # Iterate over DataFrame rows
    for i, row in df.iterrows():
        # Get current away team, home team and actual stadium timezone
        away_team = row['Away Team']
        home_team = row['Home Team']
        actual_stadium_timezone = row['Actual Stadium TimeZone']

        # Check if this is not the away team's first game
        if away_team in last_game_timezone:
            # If not, append last game's actual stadium timezone to list
            last_game_timezones_away.append(last_game_timezone[away_team])
        else:
            # If it is, append None (or any other value indicating no previous game)
            last_game_timezones_away.append(None)

        # Check if this is not the home team's first game
        if home_team in last_game_timezone:
            # If not, append last game's actual stadium timezone to list
            last_game_timezones_home.append(last_game_timezone[home_team])
        else:
            # If it is, append None (or any other value indicating no previous game)
            last_game_timezones_home.append(None)

        # Update last game's actual stadium timezone for current away and home teams
        last_game_timezone[away_team] = actual_stadium_timezone
        last_game_timezone[home_team] = actual_stadium_timezone

    # Add new columns to DataFrame
    df['Away Previous Game Actual Stadium TimeZone'] = last_game_timezones_away
    df['Home Previous Game Actual Stadium TimeZone'] = last_game_timezones_home

    # Add new column to DataFrame
    df['Home Previous Game Actual Stadium TimeZone'] = last_game_timezones_home
    df['Away Weekly Timezone Difference'] = df.apply(lambda row: calculate_hours_difference(row['Away Previous Game Actual Stadium TimeZone'], row['Actual Stadium TimeZone']) if pd.notnull(row['Away Previous Game Actual Stadium TimeZone']) and row['Away Previous Game Actual Stadium TimeZone'].strip() != '' else None, axis=1)
    df['Home Weekly Timezone Difference'] = df.apply(lambda row: calculate_hours_difference(row['Home Previous Game Actual Stadium TimeZone'], row['Actual Stadium TimeZone']) if pd.notnull(row['Home Previous Game Actual Stadium TimeZone']) and row['Home Previous Game Actual Stadium TimeZone'].strip() != '' else None, axis=1)

    df['Adjusted Away Timezone Change'] = df.apply(lambda row: 0 if row['Away Previous Game Actual Stadium TimeZone'] == row['Actual Stadium TimeZone'] and row['Actual Stadium'] != row['Away Stadium'] else calculate_hours_difference(row['Away Stadium TimeZone'], row['Actual Stadium TimeZone']), axis=1)
    df['Adjusted Home Timezone Change'] = df.apply(lambda row: 0 if row['Home Previous Game Actual Stadium TimeZone'] == row['Actual Stadium TimeZone'] and row['Actual Stadium'] != row['Home Stadium'] else calculate_hours_difference(row['Home Stadium TimeZone'], row['Actual Stadium TimeZone']), axis=1)

    df['Away Timezone Advantage'] = df.apply(lambda row: 0 if row['Adjusted Away Timezone Change'] == 0 else row['Adjusted Away Timezone Change'] - row['Adjusted Home Timezone Change'], axis=1)
    df['Home Timezone Advantage'] = df.apply(lambda row: 0 if row['Adjusted Home Timezone Change'] == 0 else row['Adjusted Home Timezone Change'] - row['Adjusted Away Timezone Change'], axis=1)

    #df['Away Timezone Advantage'] = (df['Away Timezone Change'] - df['Home Timezone Change'])
    #df['Home Timezone Advantage'] = (df['Home Timezone Change'] - df['Away Timezone Change'])

    df['Away Team Preseason Rank'] = df['Away Team'].map(lambda team: stadiums[team][5] if team in stadiums else 'NA')
    df['Home Team Preseason Rank'] = df['Home Team'].map(lambda team: stadiums[team][5] if team in stadiums else 'NA')

    df['Preseason Winner'] = df.apply(lambda row: row['Away Team'] if row['Away Team Preseason Rank'] < row['Home Team Preseason Rank'] else (row['Home Team'] if row['Away Team Preseason Rank'] > row['Home Team Preseason Rank'] else 'Tie'), axis=1)
    df['Preseason Difference'] = abs(df['Away Team Preseason Rank'] - df['Home Team Preseason Rank'])

    df['Away Team Adjusted Preseason Rank'] = df['Away Team'].map(lambda team: stadiums[team][5]) + np.where((df['Away Travel Advantage'] < -100) & (df['Home Stadium'] == df['Actual Stadium']), 1.5, 0) - pd.to_numeric(df['Away Timezone Advantage'], errors='coerce').fillna(0) - pd.to_numeric(df['Weekly Away Rest Advantage'], errors='coerce').fillna(0) - .125*df['Away Team Current Week Cumulative Rest Advantage']
    df['Home Team Adjusted Preseason Rank'] = df['Home Team'].map(lambda team: stadiums[team][5]) - np.where((df['Away Travel Advantage'] < -100) & (df['Home Stadium'] == df['Actual Stadium']), 1.5, 0) - pd.to_numeric(df['Home Timezone Advantage'], errors='coerce').fillna(0) - pd.to_numeric(df['Weekly Home Rest Advantage'], errors='coerce').fillna(0) - .125*df['Home Team Current Week Cumulative Rest Advantage']

    df['Adjusted Preseason Winner'] = df.apply(lambda row: row['Away Team'] if row['Away Team Adjusted Preseason Rank'] < row['Home Team Adjusted Preseason Rank'] else (row['Home Team'] if row['Away Team Adjusted Preseason Rank'] > row['Home Team Adjusted Preseason Rank'] else 'Tie'), axis=1)
    df['Adjusted Preseason Difference'] = abs(df['Away Team Adjusted Preseason Rank'] - df['Home Team Adjusted Preseason Rank'])

    df['Away Team Current Rank'] = df['Away Team'].map(lambda team: stadiums[team][6] if team in stadiums else 'NA')
    df['Home Team Current Rank'] = df['Home Team'].map(lambda team: stadiums[team][6] if team in stadiums else 'NA')

    df['Current Winner'] = df.apply(lambda row: row['Away Team'] if row['Away Team Current Rank'] < row['Home Team Current Rank'] else (row['Home Team'] if row['Away Team Current Rank'] > row['Home Team Current Rank'] else 'Tie'), axis=1)
    df['Current Difference'] = abs(df['Away Team Current Rank'] - df['Home Team Current Rank'])

    df['Away Team Adjusted Current Rank'] = df['Away Team'].map(lambda team: stadiums[team][6]) + np.where((df['Away Travel Advantage'] < -100) & (df['Home Stadium'] == df['Actual Stadium']), 1.5, 0) - pd.to_numeric(df['Away Timezone Advantage'], errors='coerce').fillna(0)-pd.to_numeric(df['Weekly Away Rest Advantage'], errors='coerce').fillna(0)-.125*df['Away Team Current Week Cumulative Rest Advantage']
    df['Home Team Adjusted Current Rank'] = df['Home Team'].map(lambda team: stadiums[team][6]) - np.where((df['Away Travel Advantage'] < -100) & (df['Home Stadium'] == df['Actual Stadium']), 1.5, 0) - pd.to_numeric(df['Home Timezone Advantage'], errors='coerce').fillna(0)-pd.to_numeric(df['Weekly Home Rest Advantage'], errors='coerce').fillna(0)-.125*df['Home Team Current Week Cumulative Rest Advantage']

    df['Adjusted Current Winner'] = df.apply(lambda row: row['Away Team'] if row['Away Team Adjusted Current Rank'] < row['Home Team Adjusted Current Rank'] else (row['Home Team'] if row['Away Team Adjusted Current Rank'] > row['Home Team Adjusted Current Rank'] else 'Tie'), axis=1)
    df['Adjusted Current Difference'] = abs(df['Away Team Adjusted Current Rank'] - df['Home Team Adjusted Current Rank'])

    df['Same Winner?'] = df.apply(lambda row: 'Same' if row['Preseason Winner'] == row['Adjusted Preseason Winner'] == row['Current Winner'] == row['Adjusted Current Winner'] else 'Different', axis=1)
    df['Home Team Winner?'] = df.apply(lambda row: 'Home Team' if row['Adjusted Current Winner'] == row['Home Team'] else 'Away Team', axis=1)
    #df['Divisional Matchup?'] = df.apply(lambda row: 'Divisional' if row['Home Team Division'] == row['Away Team Division'] else 'Non-divisional', axis=1)
    df['Divisional Matchup?'] = (df['Home Team Division'] == df['Away Team Division']).astype(int)


    # Create "HT 3 games in 10 days" and "AT 3 games in 10 Days" columns with default "No"
    df['Home Team 3 games in 10 days'] = 'No'
    df['Away Team 3 games in 10 days'] = 'No'

    # Convert 'Date' column to datetime objects
    df['Date'] = pd.to_datetime(df['Date'])

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the home and away teams
        home_team = row['Home Team']
        away_team = row['Away Team']
        game_date = row['Date']

        # Calculate the start date of the 10-day window
        ten_days_ago = game_date - pd.Timedelta(days=10)

        # Get the previous 10 days of games for the home team (regardless of home/away)
        home_team_games = df[
            ((df['Home Team'] == home_team) | (df['Away Team'] == home_team)) &
            (df['Date'] >= ten_days_ago) & (df['Date'] <= game_date) 
        ].sort_values('Date', ascending=False).head(10)

        # Get the previous 10 days of games for the away team (regardless of home/away)
        away_team_games = df[
            ((df['Home Team'] == away_team) | (df['Away Team'] == away_team)) &
            (df['Date'] >= ten_days_ago) & (df['Date'] <= game_date)
        ].sort_values('Date', ascending=False).head(10)

        # Check if home team has played 3 games in the last 10 days (regardless of home/away)
        if len(home_team_games) >= 3:
            df.loc[index, 'Home Team 3 games in 10 days'] = 'Yes'

        # Check if away team has played 3 games in the last 10 days (regardless of home/away)
        if len(away_team_games) >= 3:
            df.loc[index, 'Away Team 3 games in 10 days'] = 'Yes'

    # Create "HT 4 games in 17 days" and "AT 4 games in 17 Days" columns with default "No"
    df['Home Team 4 games in 17 days'] = 'No'
    df['Away Team 4 games in 17 days'] = 'No'

    # Convert 'Date' column to datetime objects
    df['Date'] = pd.to_datetime(df['Date'])

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the home and away teams
        home_team = row['Home Team']
        away_team = row['Away Team']
        game_date = row['Date']

        # Calculate the start date of the 10-day window
        seventeen_days_ago = game_date - pd.Timedelta(days=17)

        # Get the previous 10 days of games for the home team (regardless of home/away)
        home_team_games = df[
            ((df['Home Team'] == home_team) | (df['Away Team'] == home_team)) &
            (df['Date'] >= seventeen_days_ago) & (df['Date'] <= game_date) 
        ].sort_values('Date', ascending=False).head(17)

        # Get the previous 10 days of games for the away team (regardless of home/away)
        away_team_games = df[
            ((df['Home Team'] == away_team) | (df['Away Team'] == away_team)) &
            (df['Date'] >= seventeen_days_ago) & (df['Date'] <= game_date)
        ].sort_values('Date', ascending=False).head(17)

        # Check if home team has played 3 games in the last 10 days (regardless of home/away)
        if len(home_team_games) >= 4:
            df.loc[index, 'Home Team 4 games in 17 days'] = 'Yes'

        # Check if away team has played 3 games in the last 10 days (regardless of home/away)
        if len(away_team_games) >= 4:
            df.loc[index, 'Away Team 4 games in 17 days'] = 'Yes'


    # Convert 'NA' to NaN
    df['Away Team Weekly Rest'] = df['Away Team Weekly Rest'].replace('NA', pd.NA)
    df['Home Team Weekly Rest'] = df['Home Team Weekly Rest'].replace('NA', pd.NA)

    # Convert to integers
    df['Away Team Weekly Rest'] = pd.to_numeric(df['Away Team Weekly Rest'], errors='coerce')
    df['Home Team Weekly Rest'] = pd.to_numeric(df['Home Team Weekly Rest'], errors='coerce')        

    df['Away Team Short Rest'] = 'No'
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the home and away teams
        home_team_rest = row['Home Team Weekly Rest']
        away_team_rest = row['Away Team Weekly Rest']
        game_date = row['Date']

        # Check for short rest and rest disadvantage
        if (away_team_rest < 7) and (away_team_rest < home_team_rest):
            # Update the 'Away Team Short Rest' for the specific row
            df.loc[index, 'Away Team Short Rest'] = 'Yes'

    #Remnove this line for the first time you run the schedule in May############################################################################################
    # Assuming your DataFrame is named 'df'
    nfl_schedule_with_odds_circa = pd.read_csv('nfl_schedule_with_odds_circa.csv')
    df = df.assign(
        **{
            "Away Team Spread": nfl_schedule_with_odds_circa["Away Team Spread"],
            "Home Team Spread": nfl_schedule_with_odds_circa["Home Team Spread"],
            "Away Team Moneyline": nfl_schedule_with_odds_circa["Away Team Moneyline"],
            "Home Team Moneyline": nfl_schedule_with_odds_circa["Home Team Moneyline"],
            "Away Team Implied Odds to Win": nfl_schedule_with_odds_circa["Away Team Implied Odds to Win"],
            "Home team Implied Odds to Win": nfl_schedule_with_odds_circa["Home team Implied Odds to Win"],
            "Away Team Fair Odds": nfl_schedule_with_odds_circa["Away Team Fair Odds"],
            "Home Team Fair Odds": nfl_schedule_with_odds_circa["Home Team Fair Odds"],
        }
    )


    # Save the DataFrame to a CSV file
    #df.to_csv('nfl_schedule_circa.csv', index=False)

    ### THIS CELL GETS THE CUMULATIVE REMAINING WIN ODDS AND FUTURE VALUE
    import pandas as pd

    # Read the CSV file
    #csv_file = "nfl_schedule_circa.csv"
    #df = pd.read_csv(csv_file)

    # Convert percentage strings to floats
    df["Away Team Fair Odds"] = df["Away Team Fair Odds"].str.rstrip("%").astype(float) / 100
    df["Home Team Fair Odds"] = df["Home Team Fair Odds"].str.rstrip("%").astype(float) / 100

    # Calculate expected win advantage for away team
    df["Away Team Expected Win Advantage"] = round(df["Away Team Fair Odds"] - 0.5,4)

    # Calculate expected win advantage for home team
    df["Home Team Expected Win Advantage"] = round(df["Home Team Fair Odds"] - 0.5,4)

    # Initialize an empty dictionary to store team information
    team_dict = {}

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        week = row["Week"]
        away_team = row["Away Team"]
        home_team = row["Home Team"]    
        away_odds = row["Away Team Expected Win Advantage"]
        home_odds = row["Home Team Expected Win Advantage"]

        # Create a nested dictionary for each team if not already present
        if away_team not in team_dict:
            team_dict[away_team] = {}
        if home_team not in team_dict:
            team_dict[home_team] = {}

        # Populate the nested dictionary with game details and odds
        team_dict[away_team][week] = {"Opponent": home_team, "Home/Away": "Away", "Win Odds": away_odds}
        team_dict[home_team][week] = {"Opponent": away_team, "Home/Away": "Home", "Win Odds": home_odds}

    # Calculate cumulative win percentage for each team
    for team, games in team_dict.items():
        for week, details in games.items():
            opponent = details["Opponent"]
            home_away = details["Home/Away"]
            win_odds = details["Win Odds"]

            # Get the remaining weeks for the team
            remaining_weeks = [w for w in games.keys() if int(w.split()[1]) > int(week.split()[1])]

            #print(remaining_weeks)

            # Calculate cumulative win percentage
            if remaining_weeks:
                cumulative_win_odds = sum(team_dict[team][w]["Win Odds"] for w in remaining_weeks)
                cumulative_win_percentage = cumulative_win_odds/len(remaining_weeks)
            else:
                cumulative_win_percentage = 0  # Set to 0 for week 18

            # Add the cumulative win percentage to the dictionary
            team_dict[team][week]["Cumulative Win Percentage"] = cumulative_win_percentage

    # Print the updated team dictionary with cumulative win percentages
    #for team, games in team_dict.items():
    #    print(f"{team}:")
    #    for week, details in games.items():
    #        opponent = details["Opponent"]
    #        home_away = details["Home/Away"]
    #        win_odds = details["Win Odds"]
    #        cumulative_win_percentage = details.get("Cumulative Win Percentage", 0)  # Default to 0 if not calculated
    #        print(f"  {week}: {opponent} ({home_away}), Win Odds: {win_odds:.2f}, Cumulative Win Percentage: {cumulative_win_percentage:.4f}")

    # Read the original CSV file
    #csv_file = "nfl_schedule_with_odds_circa.csv"
    #df = pd.read_csv(csv_file)

    # Initialize empty lists for cumulative win percentages
    away_cumulative_win_percentages = []
    home_cumulative_win_percentages = []

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        week = row["Week"]
        away_team = row["Away Team"]
        home_team = row["Home Team"]

        # Get cumulative win percentages from your dictionary
        away_cumulative_win_percentage = team_dict.get(away_team, {}).get(week, {}).get("Cumulative Win Percentage", 0)
        home_cumulative_win_percentage = team_dict.get(home_team, {}).get(week, {}).get("Cumulative Win Percentage", 0)

        # Append to the lists
        away_cumulative_win_percentages.append(away_cumulative_win_percentage)
        home_cumulative_win_percentages.append(home_cumulative_win_percentage)

    # Add new columns to the DataFrame
    df["Away Team Cumulative Win Percentage"] = away_cumulative_win_percentages
    df["Home Team Cumulative Win Percentage"] = home_cumulative_win_percentages

    # Save the updated DataFrame to a new CSV file
    #output_csv_file = "nfl_schedule_with_cumulative_win_percentages_circa.csv"
    #df.to_csv(output_csv_file, index=False)

    # Read the original CSV file
    #csv_file = "nfl_schedule_with_cumulative_win_percentages_circa.csv"
    #df = pd.read_csv(csv_file)

    # Define a function to calculate the "Away Team Future Value"
    #def calculate_away_team_future_value(cumulative_advantage):
    #    thresholds = [0.11, 0.09, 0.07, 0.05, 0.03, 0.01, -0.01, -0.03, -0.05, -0.07, -0.09, -0.11]
    #    values = [5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, 0.5, 0, 0]

    #    for i in range(len(thresholds)):
    #        if cumulative_advantage >= thresholds[i]:
    #            return values[i]

        # If cumulative_advantage is less than -0.11, return 0
    #    return 0
    # Define a function to calculate the "Away Team Future Value"
    #def calculate_home_team_future_value(cumulative_advantage):
    #    thresholds = [0.11, 0.09, 0.07, 0.05, 0.03, 0.01, -0.01, -0.03, -0.05, -0.07, -0.09, -0.11]
    #    values = [5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, 0.5, 0, 0]

    #    for i in range(len(thresholds)):
    #        if cumulative_advantage >= thresholds[i]:
    #            return values[i]

        # If cumulative_advantage is less than -0.11, return 0
    #    return 0

    # Apply the function to create the new column
    #df["Away Team Future Value"] = df["Away Team Cumulative Win Percentage"].apply(calculate_away_team_future_value)
    #df["Home Team Future Value"] = df["Home Team Cumulative Win Percentage"].apply(calculate_home_team_future_value)

    # Save the updated DataFrame to a new CSV file
    #output_csv_file = "nfl_schedule_with_future_values_circa.csv"
    #df.to_csv(output_csv_file, index=False)

    #import pandas as pd

    # Read the original CSV file
    #csv_file = "nfl_schedule_with_cumulative_win_percentages_circa.csv"
    #df = pd.read_csv(csv_file)

    # Get unique week values
    unique_weeks = df["Week"].unique()

    # Create an empty DataFrame to store the consolidated data
    consolidated_df = pd.DataFrame()

    # Calculate the maximum cumulative win percentage for each week
    max_cumulative_win_percentage = {}
    for week in unique_weeks:
        week_df = df[df["Week"] == week]
        max_cumulative_win_percentage[week] = max(week_df["Away Team Cumulative Win Percentage"].max(),
                                                  week_df["Home Team Cumulative Win Percentage"].max())

    # Calculate the minimum cumulative win percentage for each week
    min_cumulative_win_percentage = {}
    for week in unique_weeks:
        week_df = df[df["Week"] == week]
        min_cumulative_win_percentage[week] = min(week_df["Away Team Cumulative Win Percentage"].min(), 
                                                  week_df["Home Team Cumulative Win Percentage"].min())

    # Calculate the range of cumulative win percentages for each week
    range_cumulative_win_percentage = {}
    for week in unique_weeks:
        range_cumulative_win_percentage[week] = max_cumulative_win_percentage[week] - min_cumulative_win_percentage[week]

    # Define a function to calculate the star rating
    def calculate_star_rating(cumulative_win_percentage, week):
        # Normalize the cumulative win percentage to a scale of 0 to 1
        try:
            normalized_percentage = (cumulative_win_percentage - min_cumulative_win_percentage[week]) / range_cumulative_win_percentage[week]
            # Assign stars linearly based on the normalized percentage
            return round(10 * normalized_percentage) / 2
        except ZeroDivisionError:
            return 0.0

    # Apply the function to create the new columns for each week

    for week in unique_weeks:
        week_df = df[df["Week"] == week]

        # Calculate star ratings first
        week_df["Away Team Star Rating"] = week_df["Away Team Cumulative Win Percentage"].apply(lambda x: calculate_star_rating(x, week))
        week_df["Home Team Star Rating"] = week_df["Home Team Cumulative Win Percentage"].apply(lambda x: calculate_star_rating(x, week))

        # Mark Thanksgiving Favorites
        # Find Week 13 games and winners
        week13_df = df[df["Week"] == "Week 13"]
        week13_winners = week13_df["Adjusted Current Winner"].unique()

        # Create new columns and mark Thanksgiving Favorites
        week_df["Away Team Thanksgiving Favorite"] = week_df.apply(
            lambda row: True
            if (1 <= int(row["Week"].replace("Week ", "")) <= 12) and (row["Away Team"] in week13_winners)
            else False,
            axis=1,
        )

        week_df["Home Team Thanksgiving Favorite"] = week_df.apply(
            lambda row: True
            if (1 <= int(row["Week"].replace("Week ", "")) <= 12) and (row["Home Team"] in week13_winners)
            else False,
            axis=1,
        )

        # Mark Christmas Favorites
        # Find Week 18 games and winners
        week18_df = df[df["Week"] == "Week 18"]
        week18_winners = week18_df["Adjusted Current Winner"].unique()

        # Create new columns and mark Thanksgiving Favorites
        week_df["Away Team Christmas Favorite"] = week_df.apply(
            lambda row: True
            if (1 <= int(row["Week"].replace("Week ", "")) <= 17) and (row["Away Team"] in week18_winners)
            else False,
            axis=1,
        )

        week_df["Home Team Christmas Favorite"] = week_df.apply(
            lambda row: True
            if (1 <= int(row["Week"].replace("Week ", "")) <= 17) and (row["Home Team"] in week18_winners)
            else False,
            axis=1,
        )

        consolidated_df = pd.concat([consolidated_df, week_df])

    # Create the 'Divisional Matchup Boolean' column
    consolidated_df["Divisional Matchup Boolean"] = 0

    # Set values based on 'Divisional Matchup?' column
    consolidated_df.loc[consolidated_df["Divisional Matchup?"] == True, "Divisional Matchup Boolean"] = 1

    # Save the consolidated DataFrame to a single CSV file
    consolidated_csv_file = "nfl_schedule_circa.csv"
    consolidated_df.to_csv(consolidated_csv_file, index=False)
    
    collect_schedule_travel_ranking_data_nfl_schedule_circa_df = consolidated_df
    
    return collect_schedule_travel_ranking_data_nfl_schedule_circa_df

In [ ]:
#THIS CELLS SCRAPES DK FOR CURRENT ODDS DO NOT RUN IN SEASON. ADJUST MANUALLY IN NEXT CELL

import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_preseason_odds():
    url = "https://sportsbook.draftkings.com/leagues/football/nfl"

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')


    team_name_mapping = {
        "ARI Cardinals" : "Arizona Cardinals",
        "ATL Falcons" : "Atlanta Falcons",
        "BAL Ravens" : "Baltimore Ravens",
        "BUF Bills" : "Buffalo Bills",
        "CAR Panthers" : "Carolina Panthers",
        "CHI Bears" : "Chicago Bears",
        "CIN Bengals" : 'Cincinnati Bengals',
        "CLE Browns" : 'Cleveland Browns',
        "DAL Cowboys" : 'Dallas Cowboys',
        "DEN Broncos" : 'Denver Broncos',
        "DET Lions" : 'Detroit Lions',
        "GB Packers" : 'Green Bay Packers',
        "HOU Texans" : 'Houston Texans',
        "IND Colts" : 'Indianapolis Colts',
        "JAX Jaguars" : 'Jacksonville Jaguars',
        "KC Chiefs" : 'Kansas City Chiefs',
        "LV Raiders" : 'Las Vegas Raiders',
        "LA Chargers" : 'Los Angeles Chargers',
        "LA Rams" : 'Los Angeles Rams',
        "MIA Dolphins" : 'Miami Dolphins',
        "MIN Vikings" : 'Minnesota Vikings',
        "NE Patriots" : 'New England Patriots',
        "NO Saints" : 'New Orleans Saints',
        "NY Giants" : 'New York Giants',
        "NY Jets" : 'New York Jets',
        "PHI Eagles" : 'Philadelphia Eagles',
        "PIT Steelers" : 'Pittsburgh Steelers',
        "SF 49ers" : 'San Francisco 49ers',
        "SEA Seahawks" : 'Seattle Seahawks',
        "TB Buccaneers" : 'Tampa Bay Buccaneers',
        "TEN Titans" : 'Tennessee Titans',
        "WAS Commanders" : 'Washington Commanders'
    }

    # Find all the table rows containing game data
    game_rows = soup.find_all('tr', class_=['break-line', ''])

    games = []
    game_data = {}  # Temporary dictionary to store game data

    for i, row in enumerate(game_rows):
        # Extract time only from the first row of a game
        if 'break-line' in row['class']:
            time = row.find('span', class_='event-cell__start-time').text
            game_data['Time'] = time

        # Extract team and odds - handle potential missing elements
        team = row.find('div', class_='event-cell__name-text')
        if team:
            team = team.text.strip()
            team = team_name_mapping.get(team, team)
        else:
            team = None  # Set team to None if not found

        odds_element = row.find('span', class_='sportsbook-odds american no-margin default-color')
        if odds_element:
            odds = odds_element.text.strip().replace('−', '-')
            odds = int(odds)
        else:
            odds = None  # Set odds to None if not found

        # Assign team and odds to the appropriate key in the game_data dictionary
        if i % 2 == 0:  # Even index: Away Team
            game_data['Away Team'] = team
            game_data['Away Odds'] = odds
        else:  # Odd index: Home Team
            game_data['Home Team'] = team
            game_data['Home Odds'] = odds

            # Append complete game data to the games list and reset game_data
            games.append(game_data)
            game_data = {}

    # Create pandas DataFrame from the extracted data
    df = pd.DataFrame(games)

    print(df)
    df.to_csv('Live Scraped Odds.csv', index=False)
    
    live_scraped_odds_nfl_df = df
    
    return live_scraped_odds_nfl_df



In [ ]:
#THIS CELL ADDS THE ODDS TO THE DF/CSV - MAKE SURE TO ADD/REMOVE MANUAL ODDS UPDATES IF THEY DON'T RENDER IN SCRAPED DF

import pandas as pd
def add_odds_to_main_csv():
    # 0: Stadium | 1: Lattitude | 2: Longitude | 3: Timezone | 4: Division | 5: Start of 2023 Season Rank | 6: Current Rank | 7: Average points better than Average Team (Used for Spread and Odds Calculation)
    stadiums = {
        'Arizona Cardinals': ['State Farm Stadium', 33.5277, -112.262608, 'America/Denver', 'NFC West', 26, 24, -.5],
        'Atlanta Falcons': ['Mercedez-Benz Stadium', 33.757614, -84.400972, 'America/New_York', 'NFC South', 13, 18, 0],
        'Baltimore Ravens': ['M&T Stadium', 39.277969, -76.622767, 'America/New_York', 'AFC North', 3, 3, 3],
        'Buffalo Bills': ['Highmark Stadium', 42.773739, -78.786978, 'America/New_York', 'AFC East', 5, 7, 3.5],
        'Carolina Panthers': ['Bank of America Stadium', 35.225808, -80.852861, 'America/New_York', 'NFC South', 32, 32, -7],
        'Chicago Bears': ['Soldier Field', 41.862306, -87.616672, 'America/Chicago', 'NFC North', 15, 19, -2],
        'Cincinnati Bengals': ['Paycor Stadium', 39.095442, -84.516039, 'America/New_York', 'AFC North', 6, 11, 2],
        'Cleveland Browns': ['Cleveland Browns Stadium', 41.506022, -81.699564, 'America/New_York', 'AFC North', 17, 20, 0],
        'Dallas Cowboys': ['AT&T Stadium', 32.747778, -97.092778, 'America/Chicago', 'NFC East', 9, 6, 1.5],
        'Denver Broncos': ['Empower Field at Mile High', 39.743936, -105.020097, 'America/Denver', 'AFC West', 29, 29, -5.5],
        'Detroit Lions': ['Ford Field', 42.340156, -83.045808, 'America/New_York', 'NFC North', 4, 5, 3],
        'Green Bay Packers': ['Lambeau Field', 44.501306, -88.062167, 'America/Chicago', 'NFC North', 10, 12, -4],
        'Houston Texans': ['NRG Stadium', 29.684781, -95.410956, 'America/Chicago', 'AFC South', 7, 8, 3.5],
        'Indianapolis Colts': ['Lucas Oil Stadium', 39.760056, -86.163806, 'America/New_York', 'AFC South', 20, 19, -2],
        'Jacksonville Jaguars': ['Everbank Stadium', 30.323925, -81.637356, 'America/New_York', 'AFC South', 18, 17, -.5],
        'Kansas City Chiefs': ['Arrowhead Stadium', 39.048786, -94.484566, 'America/Chicago', 'AFC West', 1, 1, 5],
        'Las Vegas Raiders': ['Allegiant Stadium', 36.090794, -115.183952, 'America/Los_Angeles', 'AFC West', 28, 26, -3],
        'Los Angeles Chargers': ['SoFi Stadium', 33.953587, -118.33963, 'America/Los_Angeles', 'AFC West', 14, 17, 1.5],
        'Los Angeles Rams': ['SoFi Stadium', 33.953587, -118.33963, 'America/Los_Angeles', 'NFC West', 16, 15, -3.5],
        'Miami Dolphins': ['Hard Rock Stadium', 25.957919, -80.238842, 'America/New_York', 'AFC East', 12, 11, -4],
        'Minnesota Vikings': ['U.S Bank Stadium', 44.973881, -93.258094, 'America/Chicago', 'NFC North', 24, 22, .5],
        'New England Patriots': ['Gillette Stadium', 42.090925, -71.26435, 'America/New_York', 'AFC East', 31, 27, -4.5],
        'New Orleans Saints': ['Caesars Superdome', 29.950931, -90.081364, 'America/Chicago', 'NFC South', 23, 16, 2],
        'New York Giants': ['MetLife Stadium', 40.812194, -74.076983, 'America/New_York', 'NFC East', 27, 31, -5],
        'New York Jets': ['MetLife Stadium', 40.812194, -74.076983, 'America/New_York', 'AFC East', 11, 13, 1],
        'Philadelphia Eagles': ['Lincoln Financial Field', 39.900775, -75.167453, 'America/New_York', 'NFC East', 8, 4, 3],
        'Pittsburgh Steelers': ['Acrisure Stadium', 40.446786, -80.015761, 'America/New_York', 'AFC North', 19, 16, .5],
        'San Francisco 49ers': ['Levi\'s Stadium', 37.713486, -122.386256, 'America/Los_Angeles', 'NFC West', 2, 1.5, 4.5],
        'Seattle Seahawks': ['Lumen Field', 47.595153, -122.331625, 'America/Los_Angeles', 'NFC West', 22, 19, .5],
        'Tampa Bay Buccaneers': ['Raymomd James Stadium', 27.975967, -82.50335, 'America/New_York', 'NFC South', 21, 21, 0],
        'Tennessee Titans': ['Nissan Stadium', 36.166461, -86.771289, 'America/Chicago', 'AFC South', 20, 24, -2.5],
        'Washington Commanders': ['FedExField', 38.907697, -76.864517, 'America/New_York', 'NFC East', 25, 28, -3.5]
    }

    # 0: Spread | 1: Favorite Odds| 2: Underdog Odds
    odds = {
        0: [-110, -110],
        .5: [-116, -104],    
        1: [-122, 101],
        1.5: [-128, 105],
        2: [-131, 108],
        2.5: [-142, 117],
        3: [-164, 135],
        3.5: [-191, 156],
        4: [-211, 171],
        4.5: [-224, 181],
        5: [-234, 188],
        5.5: [-244, 195],
        6: [-261, 208],
        6.5: [-282, 224],
        7: [-319, 249],
        7.5: [-346, 268],
        8: [-366, 282],
        8.5: [-397, 302],
        9: [-416, 314],
        9.5: [-436, 327],
        10: [-483, 356],
        10.5: [-538, 389],
        11: [-567, 406],
        11.5: [-646, 450],
        12: [-660, 458],
        12.5: [-675, 466],
        13: [-729, 494],
        13.5: [-819, 539],
        14: [-890, 573],
        14.5: [-984, 615],
        15: [-1134, 677],
        15.5: [-1197, 702],
        16: [-1266, 728],
        16.5: [-1267, 728],
        17: [-1381, 769],
        17.5: [-1832, 906],
        18: [-2149, 986],
        18.5: [-2590, 1079],
        19: [-3245, 1190],
        19.5: [-4323, 1324],
        20: [-4679, 1359],
        20.5: [-5098, 1396],
        21: [-5597, 1434]
    }

    df = live_scraped_odds_df

    # Create the mask for where there is no 'Home Odds'
    mask = df['Home Odds'] == ''
    # Only apply calculations if the 'Home Odds' column is empty
    if mask.any():
        # Adjust Average Points Difference for Favorite/Underdog Determination
        df['Adjusted Home Points'] = df.apply(lambda row: stadiums[row['Home Team']][7] + 1, axis=1)
        df['Adjusted Away Points'] = df.apply(lambda row: stadiums[row['Away Team']][7] - 1, axis=1)

        df['Spread'] = df.apply(lambda row: abs(stadiums[row['Away Team']][7] - stadiums[row['Home Team']][7]), axis=1)

        # Determine Favorite and Underdog
        df['Favorite'] = df.apply(lambda row: row['Home Team'] if row['Adjusted Home Points'] > row['Adjusted Away Points'] else row['Away Team'], axis=1)
        df['Underdog'] = df.apply(lambda row: row['Home Team'] if row['Adjusted Home Points'] < row['Adjusted Away Points'] else row['Away Team'], axis=1)

        # Adjust Spread based on Favorite
        df['Adjusted Spread'] = df.apply(lambda row: row['Spread'] + 2 if row['Favorite'] == row['Home Team'] else row['Spread'] - 2, axis=1)

        # Overwrite Odds based on Spread and Favorite/Underdog
        df['Home Odds'] = df.apply(lambda row: odds[row['Adjusted Spread']][0] if row['Favorite'] == row['Home Team'] else odds[row['Adjusted Spread']][1], axis=1)
        df['Away Odds'] = df.apply(lambda row: odds[row['Adjusted Spread']][1] if row['Favorite'] == row['Home Team'] else odds[row['Adjusted Spread']][0], axis=1)

    # Update the Current Week Odds Manually -- DO THIS IN THE LIVE SCRAPED ODDS ORDER. NOT THE NFL SCHEDULE DF
#    df.loc[16, 'Away Odds'] = 118.0  # Remember Python uses 0-based indexing
#    df.loc[16, 'Home Odds'] = -136.0

#    df.loc[25, 'Away Odds'] = 155.0  # Remember Python uses 0-based indexing
#    df.loc[25, 'Home Odds'] = -175.0

#    df.loc[24, 'Away Odds'] = -151.0  # Remember Python uses 0-based indexing
#    df.loc[24, 'Home Odds'] = 132.0

#    df.loc[17, 'Away Odds'] = 374.0  # Remember Python uses 0-based indexing
#    df.loc[17, 'Home Odds'] = -450.0

#    df.loc[22, 'Away Odds'] = -263.0  # Remember Python uses 0-based indexing
#    df.loc[22, 'Home Odds'] = 227.0

#    df.loc[20, 'Away Odds'] = 248.0  # Remember Python uses 0-based indexing
#    df.loc[20, 'Home Odds'] = -290.0

#    df.loc[26, 'Away Odds'] = 107.0  # Remember Python uses 0-based indexing
#    df.loc[26, 'Home Odds'] = -125.0

#    df.loc[21, 'Away Odds'] = -180.0  # Remember Python uses 0-based indexing
#    df.loc[21, 'Home Odds'] = 160.0

#    df.loc[18, 'Away Odds'] = -260.0  # Remember Python uses 0-based indexing
#    df.loc[18, 'Home Odds'] = 224.0

#    df.loc[23, 'Away Odds'] = -188.0  # Remember Python uses 0-based indexing
#    df.loc[23, 'Home Odds'] = 166.0

#    df.loc[19, 'Away Odds'] = 309.0  # Remember Python uses 0-based indexing
#    df.loc[19, 'Home Odds'] = -365.0

#    df.loc[27, 'Away Odds'] = 101.0  # Remember Python uses 0-based indexing
#    df.loc[27, 'Home Odds'] = -118.0

#    df.loc[29, 'Away Odds'] = 205.0  # Remember Python uses 0-based indexing
#    df.loc[29, 'Home Odds'] = -235.0

#    df.loc[28, 'Away Odds'] = -140.0  # Remember Python uses 0-based indexing
#    df.loc[28, 'Home Odds'] = 122.0

#    df.loc[30, 'Away Odds'] = 234.0  # Remember Python uses 0-based indexing
#    df.loc[30, 'Home Odds'] = -274.0

#    df.loc[31, 'Away Odds'] = 240.0  # Remember Python uses 0-based indexing
#    df.loc[31, 'Home Odds'] = -280.0


    #df.to_csv('TEST Manual Odds.csv', index = False)
    # Load the CSV data
    csv_df = collect_schedule_travel_ranking_data_df

    # Update CSV data with scraped odds
    for index, row in csv_df.iterrows():
        matching_row = df[
            (df['Away Team'] == row['Away Team']) & (df['Home Team'] == row['Home Team'])
        ]
        if not matching_row.empty:
            csv_df.loc[index, 'Away Team Moneyline'] = matching_row.iloc[0]['Away Odds']
            csv_df.loc[index, 'Home Team Moneyline'] = matching_row.iloc[0]['Home Odds']
    # Calculate Implied Odds and Fair Odds
    for index, row in csv_df.iterrows():
        # Implied Odds
        if row['Away Team Moneyline'] > 0:
            csv_df.loc[index, 'Away Team Implied Odds to Win'] = 100 / (row['Away Team Moneyline'] + 100)
        else:
            csv_df.loc[index, 'Away Team Implied Odds to Win'] = abs(row['Away Team Moneyline']) / (abs(row['Away Team Moneyline']) + 100)

        if row['Home Team Moneyline'] > 0:
            csv_df.loc[index, 'Home team Implied Odds to Win'] = 100 / (row['Home Team Moneyline'] + 100)
        else:
            csv_df.loc[index, 'Home team Implied Odds to Win'] = abs(row['Home Team Moneyline']) / (abs(row['Home Team Moneyline']) + 100)

        # Fair Odds
        away_implied_odds = csv_df.loc[index, 'Away Team Implied Odds to Win']
        home_implied_odds = csv_df.loc[index, 'Home team Implied Odds to Win']
        csv_df.loc[index, 'Away Team Fair Odds'] = away_implied_odds / (away_implied_odds + home_implied_odds)
        csv_df.loc[index, 'Home Team Fair Odds'] = home_implied_odds / (away_implied_odds + home_implied_odds)

        # Convert to percentage and round to 2 decimal places
        csv_df.loc[index, 'Away Team Implied Odds to Win'] = round(csv_df.loc[index, 'Away Team Implied Odds to Win'], 4)
        csv_df.loc[index, 'Home team Implied Odds to Win'] = round(csv_df.loc[index, 'Home team Implied Odds to Win'], 4)
        csv_df.loc[index, 'Away Team Fair Odds'] = round(csv_df.loc[index, 'Away Team Fair Odds'], 4)
        csv_df.loc[index, 'Home Team Fair Odds'] = round(csv_df.loc[index, 'Home Team Fair Odds'], 4)
    # Save the updated CSV
    csv_df.to_csv('nfl_schedule_circa.csv', index=False)
    main_df_with_odds_df = csv_df
    return main_df_with_odds_df

In [ ]:
#This Cell Uses original training datato determine Predicted pick percentages

#THIS CELL PREDICTS THE PICK PERCENTAGES

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def get_predicted_pick_percentages(pd):
    # Load your historical data (replace 'historical_pick_data_FV_circa.csv' with your actual file path)
    df = pd.read_csv('Circa_historical_data.csv')
    df.rename(columns={"Week": "Date"}, inplace=True)
    # Remove percentage sign and convert to float
    #df['Win %'] = df['Win %'].str.rstrip('%').astype(float) / 100
    #df['Pick %'] = df['Pick %'].str.rstrip('%').astype(float) / 100
    # Extract the numeric part (week number)
    #df['Week'] = df['Week'].str.extract(r'(\d+)').astype(int)
    #print(df['Date'])
    df['Pick %'].fillna(0.0, inplace=True)

    #print(df)
    # Split data into input features (X) and target variable (y)
    X = df[['Win %', 'Future Value (Stars)', 'Date', 'Away Team', 'Divisional Matchup?']]
    y = df['Pick %']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the Random Forest model
    rf_model = RandomForestRegressor(n_estimators=50, random_state=0)
    rf_model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = rf_model.predict(X_test)

    # Evaluate model performance (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    #print(f"Mean Absolute Error: {mae:.2f}")

    # Save the trained model (optional)
    # joblib.dump(model, 'random_forest_model.pkl')

    # Now you can use the trained model for predictions


    import pandas as pd

    # Read the CSV file into a DataFrame
    
    df = schedule_df_with_odds_df

    # Create a new DataFrame with selected columns
    selected_columns = ['Week', 'Away Team', 'Home Team', 'Away Team Fair Odds',
                        'Home Team Fair Odds', 'Away Team Star Rating', 'Home Team Star Rating', 'Divisional Matchup Boolean', 'Away Team Thanksgiving Favorite', 'Home Team Thanksgiving Favorite', 'Away Team Christmas Favorite', 'Home Team Christmas Favorite']
    new_df = df[selected_columns]

    # Read the original CSV file into a DataFrame
    #csv_path = 'nfl_Schedule_circa.csv'
    #df = pd.read_csv(csv_path)

    # Create the new DataFrame with modified column names
    away_df_circa = new_df.rename(columns={
        'Week': 'Date',
        'Away Team': 'Team',
        'Home Team': 'Opponent',
        'Away Team Fair Odds': 'Win %',
        'Away Team Star Rating': 'Future Value (Stars)',
        'Divisional Matchup Boolean': 'Divisional Matchup?'
    })
    away_df_circa['Year'] = 2024
    away_df_circa['Home/Away'] = 'Away'
    away_df_circa['Away Team'] = 1
    # Add the "Pick %" and "EV" columns (initially empty)
    away_df_circa['Pick %'] = None
    away_df_circa['EV'] = None

    # Drop the unwanted columns
    away_df_circa.drop(columns=['Home Team Fair Odds', 'Home Team Star Rating', 'Home Team Thanksgiving Favorite', 'Home Team Christmas Favorite'], inplace=True)

    # Reorder the columns
    column_order = ['EV', 'Win %', 'Pick %', 'Team', 'Opponent', 'Future Value (Stars)', 'Year', 'Date', 'Home/Away', 'Away Team', 'Divisional Matchup?', 'Away Team Thanksgiving Favorite', 'Away Team Christmas Favorite']
    away_df_circa = away_df_circa[column_order]


    # Create the new DataFrame with modified column names
    home_df_circa = new_df.rename(columns={
        'Week': 'Date',
        'Home Team': 'Team',
        'Away Team': 'Opponent',
        'Home Team Fair Odds': 'Win %',
        'Home Team Star Rating': 'Future Value (Stars)',
        'Divisional Matchup Boolean': 'Divisional Matchup?'
    })
    home_df_circa['Year'] = 2024
    home_df_circa['Home/Away'] = 'Home'
    home_df_circa['Away Team'] = 0
    # Add the "Pick %" and "EV" columns (initially empty)
    home_df_circa['Pick %'] = None
    home_df_circa['EV'] = None

    # Drop the unwanted columns
    home_df_circa.drop(columns=['Away Team Fair Odds', 'Away Team Star Rating', 'Away Team Thanksgiving Favorite', 'Away Team Christmas Favorite'], inplace=True)

    # Reorder the columns
    column_order = ['EV', 'Win %', 'Pick %', 'Team', 'Opponent', 'Future Value (Stars)', 'Year', 'Date', 'Home/Away', 'Away Team', 'Divisional Matchup?', 'Home Team Thanksgiving Favorite', 'Home Team Christmas Favorite']
    home_df_circa = home_df_circa[column_order]

    # Now `away_df_circa` contains the desired columns with modified names
    #print(home_df_circa)
    home_df_circa['Date'] = home_df_circa['Date'].str.extract(r'(\d+)').astype(int)
    away_df_circa['Date'] = away_df_circa['Date'].str.extract(r'(\d+)').astype(int)

    #print(home_df_circa)
    #print(away_df_circa)

    predictions = rf_model.predict(away_df_circa[['Win %', 'Future Value (Stars)', 'Date', 'Away Team', 'Divisional Matchup?']])
    away_df_circa['Pick %'] = predictions
    #away_df_circa.to_csv('predicted_away_data_circa.csv', index=False)

    predictions = rf_model.predict(home_df_circa[['Win %', 'Future Value (Stars)', 'Date', 'Away Team', 'Divisional Matchup?']])
    home_df_circa['Pick %'] = predictions
    #home_df_circa.to_csv('predicted_home_data_circa.csv', index=False)

    pick_predictions_df = pd.concat([away_df_circa, home_df_circa], ignore_index=True)

    # Adjust pick percentages for Thanksgiving Favorites
    pick_predictions_df["Pick %"] = pick_predictions_df.apply(
        lambda row: row["Pick %"] / 4 if row["Home Team Thanksgiving Favorite"] else row["Pick %"],
        axis=1
    )

    pick_predictions_df["Pick %"] = pick_predictions_df.apply(
        lambda row: row["Pick %"] / 4 if row["Away Team Thanksgiving Favorite"] else row["Pick %"],
        axis=1
    )

    # Adjust pick percentages for Thanksgiving Favorites
    pick_predictions_df["Pick %"] = pick_predictions_df.apply(
        lambda row: row["Pick %"] / 4 if row["Home Team Christmas Favorite"] else row["Pick %"],
        axis=1
    )

    pick_predictions_df["Pick %"] = pick_predictions_df.apply(
        lambda row: row["Pick %"] / 4 if row["Away Team Christmas Favorite"] else row["Pick %"],
        axis=1
    )

    pick_predictions_df.drop(columns=['Away Team Thanksgiving Favorite', 'Away Team Christmas Favorite', 'Home Team Thanksgiving Favorite', 'Home Team Christmas Favorite'], inplace=True)

    # Calculate the sum of "Pick %" for each date
    sum_by_date = pick_predictions_df.groupby('Date')['Pick %'].sum()

    # Update the "Pick %" column by dividing each value by the corresponding sum
    pick_predictions_df['Pick %'] = pick_predictions_df.apply(lambda row: row['Pick %'] / sum_by_date[row['Date']], axis=1)

    pick_predictions_df.to_csv('pick_predictions_df.csv', index = False)

    # Filter the DataFrame based on the "Home/Away" column
    home_df_circa = pick_predictions_df[pick_predictions_df["Home/Away"] == "Home"]
    away_df_circa = pick_predictions_df[pick_predictions_df["Home/Away"] == "Away"]


    home_df_circa = home_df_circa.rename(columns={
        'Date': 'Week',
        'Team': 'Home Team',
        'Opponent': 'Away Team',
        'Win %': 'Home Team Fair Odds',
        'Future Value (Stars)': 'Home Team Star Rating',
        "Pick %": "Home Pick %",
        "Away Team": "Home Away Matchup",
        "Divisional Matchup?": "Home Divisional Matchup Boolean"
    })

    away_df_circa = away_df_circa.rename(columns={
        'Date': 'Week',
        'Team': 'Away Team',
        'Opponent': 'Home Team',
        'Win %': 'Away Team Fair Odds',
        'Future Value (Stars)': 'Away Team Star Rating',
        "Pick %": "Away Pick %",
        "Away Team": "Away Away Matchup",
        "Divisional Matchup?": "Away Divisional Matchup Boolean"
    })


    # Drop the redundant columns from the merged DataFrame
    away_df_circa.drop(columns=['EV', 'Home/Away', 'Away Team Star Rating', 'Away Team Fair Odds', 'Year'], inplace=True)
    home_df_circa.drop(columns=['EV', 'Home/Away', 'Home Team Star Rating', 'Home Team Fair Odds', 'Year'], inplace=True)

    #print(home_df_circa)
    #print(away_df_circa)

    nfl_schedule_circa_df = pd.read_csv('nfl_schedule_circa.csv')
    nfl_schedule_circa_df['Week'] = nfl_schedule_circa_df['Week'].str.extract(r'(\d+)').astype(int)
    # Merge the DataFrames based on matching columns
    nfl_schedule_circa_df = pd.merge(nfl_schedule_circa_df, away_df_circa, 
                               left_on=['Week', 'Away Team', 'Home Team'],
                               right_on=['Week', 'Away Team', 'Home Team'],
                               how='left')
    nfl_schedule_circa_df = pd.merge(nfl_schedule_circa_df, home_df_circa, 
                               left_on=['Week', 'Away Team', 'Home Team'],
                               right_on=['Week', 'Away Team', 'Home Team'],
                               how='left')

    #print(nfl_schedule_circa_df)

    # Add 'Home Team EV' and 'Away Team EV' columns to nfl_schedule_circa_df
    nfl_schedule_circa_df['Home Team EV'] = 0.0  # Initialize with 0.0
    nfl_schedule_circa_df['Away Team EV'] = 0.0  # Initialize with 0.0


    nfl_schedule_circa_df.to_csv("Circa_Predicted_Pick_%.csv", index=False)
    return nfl_schedule_circa_df

In [ ]:
#MANUALLY ADJUST PICK PERCENTAGES
def manually_adjust_pick_predictions():
    nfl_schedule_circa_df = nfl_schedule_circa_pick_percentages_df

    # Update the Away Team Odds in row 13
    #df.loc[16, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[16, 'Home Pick %'] = 

    #df.loc[17, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[19, 'Home Pick %'] = .29

    #df.loc[20, 'Away Pick %'] = .18  # Remember Python uses 0-based indexing
    #df.loc[18, 'Home Pick %'] = 

    #df.loc[19, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[31, 'Home Pick %'] = .14

    #df.loc[26, 'Home Pick %'] = .09  # Remember Python uses 0-based indexing
    #df.loc[20, 'Home Pick %'] = 

    #df.loc[21, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[24, 'Away Pick %'] = .04

    #df.loc[22, 'Away Pick %'] =  # Remember Python uses 0-based indexing
    #df.loc[22, 'Home Pick %'] = 

    #df.loc[17, 'Home Pick %'] = .04  # Remember Python uses 0-based indexing
    #df.loc[23, 'Home Pick %'] = 

    #df.loc[18, 'Away Pick %'] = .02  # Remember Python uses 0-based indexing
    #df.loc[24, 'Home Pick %'] = 

    #df.loc[29, 'Away Pick %'] = .02  # Remember Python uses 0-based indexing
    #df.loc[25, 'Home Pick %'] = 

    #df.loc[26, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[25, 'Away Pick %'] = .02

    #df.loc[27, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[27, 'Home Pick %'] = 

    #df.loc[28, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[23, 'Away Pick %'] = .02

    #df.loc[21, 'Home Pick %'] = .01  # Remember Python uses 0-based indexing
    #df.loc[29, 'Home Pick %'] = 

    #df.loc[30, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[22, 'Home Pick %'] = .01

    #df.loc[31, 'Away Pick %'] =   # Remember Python uses 0-based indexing
    #df.loc[31, 'Home Pick %'] = .05

    nfl_schedule_circa_df.to_csv("Circa_Predicted_Pick_%.csv", index=False)
    
    return nfl_schedule_circa_df

In [ ]:
#THIS CELL FINDS EV

import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm

def calculate_ev():
    def calculate_all_scenarios(week_df):
        """
        Calculates the expected value (EV) for each team in a given week,
        considering all possible game outcomes. EV is calculated as the 
        inverse of the survival probability of all teams in that scenario.

        Args:
            week_df: DataFrame for a single week, including 'Home Team', 
                     'Away Team', 'Home Team Fair Odds', 'Away Team Fair Odds', 
                     'Home Pick %', and 'Away Pick %' columns.

        Returns:
            A DataFrame with EVs for all scenarios, all outcomes, and scenario weights.
        """

        def generate_outcomes(games):
            if not games:
                return [[]]
            else:
                outcomes = []
                for outcome in ['Home Win', 'Away Win']:
                    new_games = games[1:]
                    new_outcomes = generate_outcomes(new_games)
                    for new_outcome in new_outcomes:
                        outcomes.append([outcome] + new_outcome)
                return outcomes

        all_outcomes = generate_outcomes(list(range(len(week_df))))
        ev_df = pd.DataFrame(columns=week_df['Home Team'].tolist() + week_df['Away Team'].tolist())

        scenario_weights = []  # Calculate scenario weights directly
        for i, outcome in enumerate(tqdm(all_outcomes, desc="Calculating Scenarios", leave=False)):  
            scenario_ev = {team: 0 for team in week_df['Home Team'].unique().tolist() + week_df['Away Team'].unique().tolist()} 
            surviving_entries = 0
            scenario_weight = 1.0  # Calculate weight for the current scenario

            # Calculate surviving entries for ALL teams in the scenario
            for j, game_outcome in enumerate(outcome):
                if game_outcome == 'Home Win':
                    winning_team = week_df.iloc[j]['Home Team']
                    surviving_entries += week_df.iloc[j]['Home Pick %']
                    scenario_weight *= week_df.iloc[j]['Home Team Fair Odds'] 
                else:
                    winning_team = week_df.iloc[j]['Away Team']
                    surviving_entries += week_df.iloc[j]['Away Pick %']
                    scenario_weight *= week_df.iloc[j]['Away Team Fair Odds']

            # Calculate EV for EACH team in the scenario
            for j, game_outcome in enumerate(outcome):
                if game_outcome == 'Home Win':
                    winning_team = week_df.iloc[j]['Home Team']
                    if surviving_entries > 0:
                        scenario_ev[winning_team] = 1 / surviving_entries
                else:
                    winning_team = week_df.iloc[j]['Away Team']
                    if surviving_entries > 0:
                        scenario_ev[winning_team] = 1 / surviving_entries
                #print(f"Winning Team: {winning_team}")
                #print(f"Surviving Entries: {surviving_entries}")
                #print(f"Scenario Weight: {scenario_weight}")
                # EV is 1/survival probability for this scenario


            for team, ev in scenario_ev.items():
                ev_df.loc[i, team] = ev

            scenario_weights.append(scenario_weight)  # Append weight for the scenario

        # Calculate weighted average EV
        weighted_avg_ev = {}
        for team in ev_df.columns:
            weighted_evs_for_team = ev_df[team] * scenario_weights 
            weighted_avg_ev[team] = sum(weighted_evs_for_team) / sum(scenario_weights)

        # Update week_df with weighted average EVs using .loc
        for i in range(len(week_df)):
            week = week_df.iloc[i]['Week']
            home_team = week_df.iloc[i]['Home Team']
            away_team = week_df.iloc[i]['Away Team']

            # Find the weighted average EV for the home team
            if home_team in weighted_avg_ev:
                week_df.loc[(week_df['Week'] == week) & (week_df['Home Team'] == home_team), 'Home Team EV'] = weighted_avg_ev[home_team]

            # Find the weighted average EV for the away team
            if away_team in weighted_avg_ev:
                week_df.loc[(week_df['Week'] == week) & (week_df['Away Team'] == away_team), 'Away Team EV'] = weighted_avg_ev[away_team]

        # Print the team names and their weighted average EVs
        for team, avg_ev in weighted_avg_ev.items():
            print(f"{team}: {avg_ev:.4f}") 

        # Return updated week_df and other values
        return week_df, all_outcomes, scenario_weights 

    # Add "Week" to the beginning of each value in the 'Week' column
    nfl_schedule_circa_df_2['Week'] = nfl_schedule_circa_df_2['Week'].apply(lambda x: f"Week {x}")

    print(nfl_schedule_circa_df_2)

    for week in tqdm(range(3, 21), desc="Processing Weeks", leave=True): #########SET THE RANGE TO (1, 21) TO PROCESS THE WHOLE SEASON, or (2,3) to process ONLY WEEK . The rest you can figure out 
        week_df = nfl_schedule_circa_df_2[nfl_schedule_circa_df_2['Week'] == f"Week {week}"]
        week_df, all_outcomes, scenario_weights = calculate_all_scenarios(week_df)

        # Update nfl_schedule_circa_df_2 using the 'update' method
        nfl_schedule_circa_df_2.update(week_df[['Home Team EV', 'Away Team EV']]) 
        #print(nfl_schedule_circa_df_2)

    #print(nfl_schedule_circa_df_2)
    nfl_schedule_circa_df_2.to_csv("NFL Schedule with full ev_circa.csv", index=False)
    return nfl_schedule_circa_df_2

In [ ]:
###THIS CELL FINDS THE OPTIMAL PICKS BASED ON INTERNAL RANKINGS WITH ALL LIMITATIONS
from ortools.linear_solver import pywraplp
import pandas as pd

def get_survivor_picks_based_on_ev():
    # Loop through 100 iterations
    for iteration in range(100):
        df = full_df_with_ev
        df['Week'] = df['Week'].str.extract('(\d+)').astype(int)



        #Number of weeks that have already been played
        weeks_completed = 0

        # Teams already picked - Team name in quotes and separated by commas
        picked_teams = []
        picked_teams = []

        # Filter out weeks that have already been played and reset index
        df = df[df['Week'] > weeks_completed].reset_index(drop=True)

        # Filter out already picked teams
        df = df[~df['Adjusted Current Winner'].isin(picked_teams)].reset_index(drop=True)
        #print(df)
        # Create the solver
        solver = pywraplp.Solver.CreateSolver('SCIP')

        # Create binary variables to represent the picks, and store them in a dictionary for easy lookup
        picks = {}
        for i in range(len(df)):
            picks[i] = solver.IntVar(0, 1, 'pick_%i' % i)

        # Add the constraints
        for week in df['Week'].unique():
            # One team per week
            solver.Add(solver.Sum([picks[i] for i in range(len(df)) if df.loc[i, 'Week'] == week]) == 1)

        for team in df['Adjusted Current Winner'].unique():
            # Can't pick a team more than once
            solver.Add(solver.Sum([picks[i] for i in range(len(df)) if df.loc[i, 'Adjusted Current Winner'] == team]) <= 1)

        for i in range(len(df)):
            # Must pick from 'Adjusted Current Winner'
            #if df.loc[i, 'Adjusted Current Winner'] != df.loc[i, 'Home Team']:
                #solver.Add(picks[i] == 0)

            # Must pick from 'Same Winner?'
            if df.loc[i, 'Same Winner?'] != 'Same':
                solver.Add(picks[i] == 0)

            # Can only pick an away team if 'Adjusted Current Difference' > 10
            if df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Adjusted Current Difference'] < 10:
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Divisional Matchup?'] == 'Divisional':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Back to Back Away Games'] == 'True':
                solver.Add(picks[i] == 0)

            # If 'Divisional Matchup?' is "Divisional", can only pick if 'Adjusted Current Difference' > 10
            if df.loc[i, 'Divisional Matchup?'] == 'Divisional' and df.loc[i, 'Adjusted Current Difference'] < 10:
                solver.Add(picks[i] == 0)
            # Constraints for short rest and 4 games in 17 days (only if team is the Adjusted Current Winner)
            if df.loc[i, 'Away Team Short Rest'] == 'Yes' and df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner']:
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Home Team 4 games in 17 days'] == 'Yes' and df.loc[i, 'Home Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Away Team 4 games in 17 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team 4 games in 17 days'] == 'Yes' and df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Home Team 4 games in 17 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Home Team 3 games in 10 days'] == 'Yes' and df.loc[i, 'Home Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Away Team 3 games in 10 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team 3 games in 10 days'] == 'Yes' and df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Home Team 3 games in 10 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Adjusted Current Winner'] == 'San Francisco 49ers' and df.loc[i, 'Week'] == 15:
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Adjusted Current Winner'] in picked_teams:
                solver.Add(picks[i] == 0)
        # Dynamically create the forbidden solution list
        forbidden_solutions_1 = []
        if iteration > 0: 
            for previous_iteration in range(iteration):
                # Load the picks from the previous iteration
                previous_picks_df = pd.read_csv(f"picks_ev_circa_{previous_iteration + 1}.csv")

                # Extract the forbidden solution for this iteration
                forbidden_solution_1 = previous_picks_df['Adjusted Current Winner'].tolist()
                forbidden_solutions_1.append(forbidden_solution_1)

        # Add constraints for all forbidden solutions
        for forbidden_solution_1 in forbidden_solutions_1:
            # Get the indices of the forbidden solution in the DataFrame
            forbidden_indices_1 = []
            for i in range(len(df)):
                week_index = df.loc[i, 'Week'] - (weeks_completed + 1)
                if week_index >= 0 and week_index < len(forbidden_solution_1):
                    if (df.loc[i, 'Adjusted Current Winner'] == forbidden_solution_1[week_index]):
                        forbidden_indices_1.append(i)

            # Add the constraint that at least one of these picks should not be selected
            solver.Add(solver.Sum([1 - picks[i] for i in forbidden_indices_1]) >= 1)


        # Add the constraint for San Francisco 49ers in week 11
    #    for i in range(len(df)):
    #        if df.loc[i, 'Home Team'] == 'San Francisco 49ers' and df.loc[i, 'Week'] == 15:
    #            solver.Add(picks[i] == 0)

        # Objective: maximize the sum of Adjusted Current Difference of each game picked
        solver.Maximize(solver.Sum([picks[i] * (df.loc[i, 'Home Team EV'] if df.loc[i, 'Adjusted Current Winner'] == df.loc[i, 'Home Team'] else df.loc[i, 'Away Team EV']) for i in range(len(df))]))

        # Solve the problem and print the solution
        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL:
            print('Solution found!')
            print('Objective value =', solver.Objective().Value())

            # Initialize sums
            sum_preseason_difference = 0
            sum_adjusted_preseason_difference = 0
            sum_current_difference = 0
            sum_adjusted_current_difference = 0
            sum_ev = 0

            # Initialize picks_df
            picks_df = pd.DataFrame(columns=df.columns)

            for i in range(len(df)):
                if picks[i].solution_value() > 0:
                    # Determine if it's a divisional game and if the picked team is the home team
                    divisional_game = '(Divisional)' if df.loc[i, 'Divisional Matchup?'] == 'Divisional' else ''
                    home_team = '(Home Team)' if df.loc[i, 'Adjusted Current Winner'] == df.loc[i, 'Home Team'] else '(Away Team)'

                    # Get differences
                    preseason_difference = df.loc[i, 'Preseason Difference']
                    adjusted_preseason_difference = df.loc[i, 'Adjusted Preseason Difference']
                    current_difference = df.loc[i, 'Current Difference']
                    adjusted_current_difference = df.loc[i, 'Adjusted Current Difference']
                    # Calculate EV for this game
                    ev = (df.loc[i, 'Home Team EV'] if df.loc[i, 'Adjusted Current Winner'] == df.loc[i, 'Home Team'] else df.loc[i, 'Away Team EV'])


                    print('Week %i: Pick %s %s %s (%i, %i, %i, %i, %.4f)' % (df.loc[i, 'Week'], df.loc[i, 'Adjusted Current Winner'], divisional_game, home_team,
                                                                       preseason_difference, adjusted_preseason_difference,
                                                                       current_difference, adjusted_current_difference, ev))

                    # Add differences to sums
                    sum_preseason_difference += preseason_difference
                    sum_adjusted_preseason_difference += adjusted_preseason_difference
                    sum_current_difference += current_difference
                    sum_adjusted_current_difference += adjusted_current_difference
                    sum_ev += ev
                    picks_df = pd.concat([picks_df, df.loc[[i]]], ignore_index=True)


            # Add row to picks_df
    #        picks_df = pd.concat([picks_df, df.loc[[i]]], ignore_index=True)
            #print(picks_df)
            # Print sums
            print('\nPreseason Difference:', sum_preseason_difference)
            print('Adjusted Preseason Difference:', sum_adjusted_preseason_difference)
            print('Current Difference:', sum_current_difference)
            print('Adjusted Current Difference:', sum_adjusted_current_difference)
            print('Total EV:', sum_ev)
        else:
            print('No solution found.')
        print("\n")

            # Save the picks to a CSV file for the current iteration
        picks_df.to_csv(f'picks_ev_circa_{iteration + 1}.csv', index=False)
        
        # Append the new forbidden solution to the list
        forbidden_solutions_1.append(picks_df['Adjusted Current Winner'].tolist())
        #print(forbidden_solutions)

In [ ]:
###THIS CELL FINDS THE OPTIMAL PICKS BASED ON INTERNAL RANKINGS WITH ALL LIMITATIONS
from ortools.linear_solver import pywraplp
import pandas as pd

def get_survivor_picks_based_on_internal_rankings():

    # Loop through 100 iterations
    for iteration in range(100):
        df = full_df_with_ev
        df['Week'] = df['Week'].str.extract('(\d+)').astype(int)



        #Number of weeks that have already been played
        weeks_completed = 0

        # Teams already picked - Team name in quotes and separated by commas
        picked_teams = []

        # Filter out weeks that have already been played and reset index
        df = df[df['Week'] > weeks_completed].reset_index(drop=True)

        # Filter out already picked teams
        df = df[~df['Adjusted Current Winner'].isin(picked_teams)].reset_index(drop=True)
        #print(df)
        # Create the solver
        solver = pywraplp.Solver.CreateSolver('SCIP')

        # Create binary variables to represent the picks, and store them in a dictionary for easy lookup
        picks = {}
        for i in range(len(df)):
            picks[i] = solver.IntVar(0, 1, 'pick_%i' % i)

        # Add the constraints
        for week in df['Week'].unique():
            # One team per week
            solver.Add(solver.Sum([picks[i] for i in range(len(df)) if df.loc[i, 'Week'] == week]) == 1)

        for team in df['Adjusted Current Winner'].unique():
            # Can't pick a team more than once
            solver.Add(solver.Sum([picks[i] for i in range(len(df)) if df.loc[i, 'Adjusted Current Winner'] == team]) <= 1)

        for i in range(len(df)):
            # Must pick from 'Adjusted Current Winner'
            #if df.loc[i, 'Adjusted Current Winner'] != df.loc[i, 'Home Team']:
                #solver.Add(picks[i] == 0)

            # Must pick from 'Same Winner?'
            if df.loc[i, 'Same Winner?'] != 'Same':
                solver.Add(picks[i] == 0)

            # Can only pick an away team if 'Adjusted Current Difference' > 10
            if df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Adjusted Current Difference'] < 10:
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Divisional Matchup?'] == 'Divisional':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Back to Back Away Games'] == 'True':
                solver.Add(picks[i] == 0)

            # If 'Divisional Matchup?' is "Divisional", can only pick if 'Adjusted Current Difference' > 10
            if df.loc[i, 'Divisional Matchup?'] == 'Divisional' and df.loc[i, 'Adjusted Current Difference'] < 10:
                solver.Add(picks[i] == 0)
            # Constraints for short rest and 4 games in 17 days (only if team is the Adjusted Current Winner)
            if df.loc[i, 'Away Team Short Rest'] == 'Yes' and df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner']:
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Home Team 4 games in 17 days'] == 'Yes' and df.loc[i, 'Home Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Away Team 4 games in 17 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team 4 games in 17 days'] == 'Yes' and df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Home Team 4 games in 17 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Home Team 3 games in 10 days'] == 'Yes' and df.loc[i, 'Home Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Away Team 3 games in 10 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Away Team 3 games in 10 days'] == 'Yes' and df.loc[i, 'Away Team'] == df.loc[i, 'Adjusted Current Winner'] and df.loc[i, 'Home Team 3 games in 10 days'] == 'No':
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Adjusted Current Winner'] == 'San Francisco 49ers' and df.loc[i, 'Week'] == 15:
                solver.Add(picks[i] == 0)
            if df.loc[i, 'Adjusted Current Winner'] in picked_teams:
                solver.Add(picks[i] == 0)
        # Dynamically create the forbidden solution list
        forbidden_solutions = []
        if iteration > 0: 
            for previous_iteration in range(iteration):
                # Load the picks from the previous iteration
                previous_picks_df = pd.read_csv(f"picks_internal_rankings_circa_{previous_iteration + 1}.csv")

                # Extract the forbidden solution for this iteration
                forbidden_solution = previous_picks_df['Adjusted Current Winner'].tolist()
                forbidden_solutions.append(forbidden_solution)

        # Add constraints for all forbidden solutions
        for forbidden_solution in forbidden_solutions:
            # Get the indices of the forbidden solution in the DataFrame
            forbidden_indices = []
            for i in range(len(df)):
                week_index = df.loc[i, 'Week'] - (weeks_completed + 1)
                if week_index >= 0 and week_index < len(forbidden_solution):
                    if (df.loc[i, 'Adjusted Current Winner'] == forbidden_solution[week_index]):
                        forbidden_indices.append(i)

            # Add the constraint that at least one of these picks should not be selected
            solver.Add(solver.Sum([1 - picks[i] for i in forbidden_indices]) >= 1)


        # Add the constraint for San Francisco 49ers in week 11
        #for i in range(len(df)):
        #    if df.loc[i, 'Home Team'] == 'San Francisco 49ers' and df.loc[i, 'Week'] == 11:
        #        solver.Add(picks[i] == 1)

        # Objective: maximize the sum of Adjusted Current Difference of each game picked
        solver.Maximize(solver.Sum([picks[i] * df.loc[i, 'Adjusted Current Difference'] for i in range(len(df))]))
        # Solve the problem and print the solution
        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL:
            print('Solution found!')
            print('Objective value =', solver.Objective().Value())

            # Initialize sums
            sum_preseason_difference = 0
            sum_adjusted_preseason_difference = 0
            sum_current_difference = 0
            sum_adjusted_current_difference = 0
            sum_ev = 0

            # Initialize picks_df
            picks_df = pd.DataFrame(columns=df.columns)

            for i in range(len(df)):
                if picks[i].solution_value() > 0:
                    # Determine if it's a divisional game and if the picked team is the home team
                    divisional_game = '(Divisional)' if df.loc[i, 'Divisional Matchup?'] == 'Divisional' else ''
                    home_team = '(Home Team)' if df.loc[i, 'Adjusted Current Winner'] == df.loc[i, 'Home Team'] else '(Away Team)'

                    # Get differences
                    preseason_difference = df.loc[i, 'Preseason Difference']
                    adjusted_preseason_difference = df.loc[i, 'Adjusted Preseason Difference']
                    current_difference = df.loc[i, 'Current Difference']
                    adjusted_current_difference = df.loc[i, 'Adjusted Current Difference']
                    # Calculate EV for this game
                    ev = (df.loc[i, 'Home Team EV'] if df.loc[i, 'Adjusted Current Winner'] == df.loc[i, 'Home Team'] else df.loc[i, 'Away Team EV'])


                    print('Week %i: Pick %s %s %s (%i, %i, %i, %i, %.4f)' % (df.loc[i, 'Week'], df.loc[i, 'Adjusted Current Winner'], divisional_game, home_team,
                                                                       preseason_difference, adjusted_preseason_difference,
                                                                       current_difference, adjusted_current_difference, ev))

                    # Add differences to sums
                    sum_preseason_difference += preseason_difference
                    sum_adjusted_preseason_difference += adjusted_preseason_difference
                    sum_current_difference += current_difference
                    sum_adjusted_current_difference += adjusted_current_difference
                    sum_ev += ev
                    picks_df = pd.concat([picks_df, df.loc[[i]]], ignore_index=True)


            # Add row to picks_df
    #        picks_df = pd.concat([picks_df, df.loc[[i]]], ignore_index=True)
            #print(picks_df)
            # Print sums
            print('\nPreseason Difference:', sum_preseason_difference)
            print('Adjusted Preseason Difference:', sum_adjusted_preseason_difference)
            print('Current Difference:', sum_current_difference)
            print('Adjusted Current Difference:', sum_adjusted_current_difference)
            print('Total EV:', sum_ev)
        else:
            print('No solution found.')
        print("\n")

            # Save the picks to a CSV file for the current iteration
        picks_df.to_csv(f'picks_internal_rankings_circa_{iteration + 1}.csv', index=False)
        # Append the new forbidden solution to the list
        forbidden_solutions.append(picks_df['Adjusted Current Winner'].tolist())
        #print(forbidden_solutions)

In [ ]:
schedule_table, schedule_rows = get_schedule() #This gets the schedule fro FFToday.com

#Run Once, unless table is not defined

In [ ]:
collect_schedule_travel_ranking_data_df = collect_schedule_travel_ranking_data(pd) #This buiklds the dataframes and CSVs with all the TRavel, Ranking, Future Value, Rest etc... data

#Run each week

In [ ]:
live_scraped_odds_df = get_preseason_odds() #This gets the preseason odds from draftkings. Do not run during season

#run each week

In [ ]:
schedule_df_with_odds_df = add_odds_to_main_csv() #This function adds draftkings odds to all current games, then uses your personal rankings to calculate spreads and odds for games further out. 

#Run Each Week

In [ ]:
nfl_schedule_circa_pick_percentages_df = get_predicted_pick_percentages(pd) #This function gathers historical data and predicts pick npercentages based on that historical data

#Run Each Week

In [ ]:
nfl_schedule_circa_df_2 = manually_adjust_pick_predictions() #This function manually adjusts pick percentages if they are way off, or later in the season and need manual curation

#ONLY run if you have manually adjusted the code for the new pick percentages

In [ ]:
full_df_with_ev = calculate_ev() #This function caluclates all EV for all weeks. Might need to add a condition for which week you are on

#Run each week and adjust which week it is

In [ ]:
get_survivor_picks_based_on_ev() #This function finds all 100 possible combinations based on EV. We should add to the UI a choice to select adn deselkect all coditions and choose which week we are in etc...

#Run each week and adjust which week it is and teams picked

In [ ]:
get_survivor_picks_based_on_internal_rankings()#This function finds all 100 possible combinations based on internal rankings. We should add to the UI a choice to select adn deselkect all coditions and choose which week we are in etc...

#Run each week and adjust which week it is and teams picked

In [113]:
import ipywidgets as widgets
from IPython.display import display

In [118]:
# Create buttons
button_schedule = widgets.Button(description="Get Schedule")

# Define function to run when a button is clicked
def on_schedule_button_clicked(b):
    schedule_table, schedule_rows = get_schedule()
# Link button click to functions
button_schedule.on_click(on_schedule_button_clicked)
# Display the buttons
display(widgets.VBox([button_schedule])) # Using VBox to vertically stack buttons